# Clasificador basado en CNN
Basado en https://github.com/satojkovic/cnn-text-classification-tf/tree/use_fasttext y https://github.com/jiegzhan/multi-class-text-classification-cnn

In [1]:
#Hacer imports y cargar stopwords y vectores entrenados
import os
import json
import time
import logging
import numpy as np
import tensorflow as tf
import pandas as pd  
from text_cnn import TextCNN
from tensorflow.contrib import learn
from sklearn.model_selection import train_test_split
from pathlib import Path
import pickle

logging.getLogger().setLevel(logging.INFO)

archivoTweets = 'tweets_limpios.csv'

params = {
            "num_epochs": 100,
            "batch_size": 50,
            "num_filters": 128,
            "filter_sizes": "3,4,5",
            "embedding_dim": 128,
            "l2_reg_lambda": 0.1,
            "evaluate_every": 100,
            "dropout_keep_prob": 0.5
        }

#Cargar stopwords
df = pd.read_csv("Stopwords.txt",header=None)
spanish_stopwords = df[0].values.tolist()
print("Stopwords cargados")

C:\Users\joax\Anaconda2\envs\py36\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Stopwords cargados


In [2]:
#Método que carga los textos pre-procesados con sus polaridades y los prepara para el entrenamiento
def cargar_datos_etiquetas(filename):
    """Carga texto y polaridad"""
    df = pd.read_csv(filename)
    selected = ['Texto', 'Polaridad']
    non_selected = list(set(df.columns) - set(selected))

    df = df.drop(non_selected, axis=1) # Elimina las columnas innecesarias
    df = df.dropna(axis=0, how='any', subset=selected) # Elimina filas con valores null
    df = df.reindex(np.random.permutation(df.index)) # Revuelve el conjunto de datos

    # Convierte las polaridades en etiquetas One-hot
    labels = sorted(list(set(df[selected[1]].tolist())))
    one_hot = np.zeros((len(labels), len(labels)), int)
    np.fill_diagonal(one_hot, 1)
    label_dict = dict(zip(labels, one_hot))

    #Crea listas con los textos y las etiquetas en formato one-hot
    x_raw = df[selected[0]].tolist()
    y_raw = df[selected[1]].apply(lambda y: label_dict[y]).tolist()
    return x_raw, y_raw, df, labels

def batch_iter(data, batch_size, num_epochs, shuffle=True):
    """
    Generates a batch iterator for a dataset.
    """
    data = np.array(data)
    data_size = len(data)
    num_batches_per_epoch = int((len(data)-1)/batch_size) + 1
    for epoch in range(num_epochs):
        # Shuffle the data at each epoch
        if shuffle:
            shuffle_indices = np.random.permutation(np.arange(data_size))
            shuffled_data = data[shuffle_indices]
        else:
            shuffled_data = data
        for batch_num in range(num_batches_per_epoch):
            start_index = batch_num * batch_size
            end_index = min((batch_num + 1) * batch_size, data_size)
            yield shuffled_data[start_index:end_index]

def train_cnn():
    #Paso 0: Cargar oraciones, etiquetas y parámetros
    x_raw, y_raw, df, labels = cargar_datos_etiquetas(archivoTweets)

    #Paso 1: Obtiene vectores de las palabras y rellena los textos para tener la misma longitud
    max_document_length = max([len(x.split(' ')) for x in x_raw])
    logging.info(" Oración más larga tiene {} palabras. Se agregan 5 para tener espacio de manipulación para nuevas oraciones".format(max_document_length))
    max_document_length += 5
    
    vocab_processor = learn.preprocessing.VocabularyProcessor(max_document_length)    
    my_file = Path("fasttext_vocabulario.dat")
    preentrenado = my_file.is_file()
    if preentrenado:
        logging.info(" Cargando vectores generados previamente")
        with open('fasttext_vocabulario.dat', 'rb') as fr:
            vocab = pickle.load(fr)
        embedding = np.load('fasttext_embeddings.npy')

        pretrain = vocab_processor.fit(vocab.keys())
        x = np.array(list(vocab_processor.transform(x_raw)))
        embedding_size = params['embedding_dim']
        vocab_size = len(vocab)
    else:
        logging.info(" Generando vectores")
        x = np.array(list(vocab_processor.fit_transform(x_raw)))
        embedding_size = params['embedding_dim']
        vocab_size = len(vocab_processor.vocabulary_)
        
    y = np.array(y_raw)

    #Paso 2: Divide el dataset original en entrenamiento y prueba
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=42)

    #Paso 3: revuelve el dataset de entrenamiento y divide el de entrenamiento en entrenamiento y validación
    shuffle_indices = np.random.permutation(np.arange(len(y_train)))
    x_shuffled = x_train[shuffle_indices]
    y_shuffled = y_train[shuffle_indices]
    x_train, x_dev, y_train, y_dev = train_test_split(x_shuffled, y_shuffled, test_size=0.1)

    #Paso 4: guarda las etiquetas en un archivo JSON: labels.json para hacer predicciones luego
    with open('labels.json', 'w') as outfile:
        json.dump(labels, outfile, indent=4)

    logging.info(' x_train: {}, x_dev: {}, x_test: {}'.format(len(x_train), len(x_dev), len(x_test)))
    logging.info(' y_train: {}, y_dev: {}, y_test: {}'.format(len(y_train), len(y_dev), len(y_test)))

    #Paso 5: Construir el grafo y el objeto CNN
    graph = tf.Graph()
    with graph.as_default():
        session_conf = tf.ConfigProto(allow_soft_placement=True, log_device_placement=False)
        sess = tf.Session(config=session_conf)
        with sess.as_default():
            cnn = TextCNN(
                sequence_length=x_train.shape[1],
                num_classes=y_train.shape[1],
                vocab_size=vocab_size,
                embedding_size=params['embedding_dim'],
                filter_sizes=list(map(int, params['filter_sizes'].split(","))),
                num_filters=params['num_filters'],
                l2_reg_lambda=params['l2_reg_lambda'],
                pre_trained=preentrenado)

            global_step = tf.Variable(0, name="global_step", trainable=False)
            optimizer = tf.train.AdamOptimizer(1e-3)
            grads_and_vars = optimizer.compute_gradients(cnn.loss)
            train_op = optimizer.apply_gradients(grads_and_vars, global_step=global_step)

            timestamp = str(int(time.time()))
            out_dir = os.path.abspath(os.path.join(os.path.curdir, "trained_model_" + timestamp))

            checkpoint_dir = os.path.abspath(os.path.join(out_dir, "checkpoints"))
            checkpoint_prefix = os.path.join(checkpoint_dir, "model")
            if not os.path.exists(checkpoint_dir):
                os.makedirs(checkpoint_dir)
            saver = tf.train.Saver(tf.global_variables())
            
            # One training step: train the model with one batch
            def train_step(x_batch, y_batch):
                if preentrenado:
                    feed_dict = {
                        cnn.input_x: x_batch,
                        cnn.input_y: y_batch,
                        cnn.dropout_keep_prob: params['dropout_keep_prob'],
                        cnn.embedding_placeholder: embedding
                    }
                else:
                    feed_dict = {
                        cnn.input_x: x_batch,
                        cnn.input_y: y_batch,
                        cnn.dropout_keep_prob: params['dropout_keep_prob']
                    }
                _, step, loss, acc = sess.run([train_op, global_step, cnn.loss, cnn.accuracy], feed_dict)

            # One evaluation step: evaluate the model with one batch
            def dev_step(x_batch, y_batch):
                if preentrenado:
                    feed_dict = {
                        cnn.input_x: x_batch,
                        cnn.input_y: y_batch,
                        cnn.dropout_keep_prob: 1.0,
                        cnn.embedding_placeholder: embedding
                    }
                else:
                    feed_dict = {
                        cnn.input_x: x_batch,
                        cnn.input_y: y_batch,
                        cnn.dropout_keep_prob: 1.0
                    }
                step, loss, acc, num_correct = sess.run([global_step, cnn.loss, cnn.accuracy, cnn.num_correct], feed_dict)
                #return num_correct
                return acc

            # Write vocabulary
            vocab_processor.save(os.path.join(out_dir, "vocab"))
            
            #Inicializa las variables del clasificador
            sess.run(tf.global_variables_initializer())

            # Training starts here
            train_batches = batch_iter(list(zip(x_train, y_train)), params['batch_size'], params['num_epochs'])
            best_accuracy, best_at_step = 0, 0

            #Paso 6: entrenar el modelo de CNN con x_train y y_train (batch por batch)
            logging.info(" Inicio de entrenamiento")
            for train_batch in train_batches:
                x_train_batch, y_train_batch = zip(*train_batch)
                train_step(x_train_batch, y_train_batch)
                current_step = tf.train.global_step(sess, global_step)

                if current_step % params['evaluate_every'] == 0:
                    logging.info(" Etapa: {}".format(current_step))
                    #Paso 6.1: evaluar el modelo con x_dev y y_dev (batch por batch)
                    #dev_batches = batch_iter(list(zip(x_dev, y_dev)), params['batch_size'], 1)
                    #total_dev_correct = 0
                    #totalBatches = 0
                    #for dev_batch in dev_batches:
                    #    x_dev_batch, y_dev_batch = zip(*dev_batch)
                    #    total_dev_correct += dev_step(x_dev_batch, y_dev_batch)
                    #    totalBatches += 1
                    #dev_accuracy = float(total_dev_correct) / totalBatches
                    
                    dev_accuracy = dev_step(x_dev, y_dev)
                    
                    #Este es para cuando se usa con la cuenta
                    #dev_accuracy = float(total_dev_correct) / len(y_dev)
                    
                    logging.critical(' Exactitud en set de validación: {}'.format(dev_accuracy))

                    if dev_accuracy >= best_accuracy:
                        #Paso 6.2: Guardar el modelo si es el mejor basado en exactitud en el set de validación
                        best_accuracy, best_at_step = dev_accuracy, current_step
                        path = saver.save(sess, checkpoint_prefix, global_step=current_step)
                        logging.critical(' Modelo guardado en {} en etapa {}'.format(path, best_at_step))
                        logging.critical(' Mejor exactitud es {} en etapa {}'.format(best_accuracy, best_at_step))

            #Paso 7: Predecir x_test (batch por batch)
            #test_batches = batch_iter(list(zip(x_test, y_test)), params['batch_size'], 1)
            #total_test_correct = 0
            #totalBatches = 0
            #for test_batch in test_batches:
            #    x_test_batch, y_test_batch = zip(*test_batch)
            #    num_test_correct = dev_step(x_test_batch, y_test_batch)
            #    total_test_correct += num_test_correct
            #    totalBatches += 1
            #test_accuracy = float(total_test_correct) / totalBatches
            test_accuracy = dev_step(x_test, y_test)
            
            #test_accuracy = float(total_test_correct) / len(y_test)
            logging.critical(' Exactitud en set de pruebas es {} basado en el mejor modelo {}'.format(test_accuracy, path))
            logging.critical(' Entrenamiento completado')

In [3]:
train_cnn()

INFO:root: Oración más larga tiene 28 palabras. Se agregan 5 para tener espacio de manipulación para nuevas oraciones


Instructions for updating:
Please use tensorflow/transform or tf.data.


Instructions for updating:
Please use tensorflow/transform or tf.data.


Instructions for updating:
Please use tensorflow/transform or tf.data.


Instructions for updating:
Please use tensorflow/transform or tf.data.
INFO:root: Cargando vectores generados previamente
C:\Users\joax\Anaconda2\envs\py36\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
INFO:summarizer.preprocessing.cleaner:'pattern' package not found; tag filters are not available for English


Instructions for updating:
Please use tensorflow/transform or tf.data.


Instructions for updating:
Please use tensorflow/transform or tf.data.
INFO:root: x_train: 5845, x_dev: 650, x_test: 722
INFO:root: y_train: 5845, y_dev: 650, y_test: 722
INFO:root: Inicio de entrenamiento
INFO:root: Etapa: 100
CRITICAL:root: Exactitud en set de validación: 0.2846153974533081
CRITICAL:root: Modelo guardado en D:\respaldo joax\UCR\Maestria computacion\2018-1\NPL\Deep learning\trained_model_1527904205\checkpoints\model-100 en etapa 100
CRITICAL:root: Mejor exactitud es 0.2846153974533081 en etapa 100
INFO:root: Etapa: 200
CRITICAL:root: Exactitud en set de validación: 0.3461538553237915
CRITICAL:root: Modelo guardado en D:\respaldo joax\UCR\Maestria computacion\2018-1\NPL\Deep learning\trained_model_1527904205\checkpoints\model-200 en etapa 200
CRITICAL:root: Mejor exactitud es 0.3461538553237915 en etapa 200
INFO:root: Etapa: 300
CRITICAL:root: Exactitud en set de validación: 0.35538461804389954
CRITICAL:root: Modelo guardado en D:\respaldo joax\UCR\Maestria computacion

CRITICAL:root: Exactitud en set de validación: 0.3799999952316284
INFO:root: Etapa: 5500
CRITICAL:root: Exactitud en set de validación: 0.39384615421295166
INFO:root: Etapa: 5600
CRITICAL:root: Exactitud en set de validación: 0.383076936006546
INFO:root: Etapa: 5700
CRITICAL:root: Exactitud en set de validación: 0.38923075795173645
INFO:root: Etapa: 5800
CRITICAL:root: Exactitud en set de validación: 0.3969230651855469
INFO:root: Etapa: 5900
CRITICAL:root: Exactitud en set de validación: 0.4046153724193573
CRITICAL:root: Modelo guardado en D:\respaldo joax\UCR\Maestria computacion\2018-1\NPL\Deep learning\trained_model_1527904205\checkpoints\model-5900 en etapa 5900
CRITICAL:root: Mejor exactitud es 0.4046153724193573 en etapa 5900
INFO:root: Etapa: 6000
CRITICAL:root: Exactitud en set de validación: 0.4107692241668701
CRITICAL:root: Modelo guardado en D:\respaldo joax\UCR\Maestria computacion\2018-1\NPL\Deep learning\trained_model_1527904205\checkpoints\model-6000 en etapa 6000
CRITIC